In [1]:
import pandas as pd

In [2]:
cases = pd.read_csv("V3_.csv")#pd.read_csv("Validated_V0_Desriptions.csv")

In [3]:
data = pd.read_csv("data/raw_data.csv", parse_dates=["block_time"])

C:\Users\Anton\AppData\Local\Temp\ipykernel_33404\1695098993.py:1: DtypeWarning: Columns (5,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/raw_data.csv", parse_dates=["block_time"])


In [4]:
data.head()

,transaction_id,transaction_time,block_time,sender_id,receiver_id,receiver_type,amount,currency,currency_hash,currency_type,amount_usd,gas_price,gas_limit,gas_used,gas_predicted,balance_in,balance_out,label,risk_level,file_name
0,0x603882c65e180a52ca883dd7817f08d2eb828bbce600...,NaN,2021-09-16 08:45:02,0xc97f82c80df57c34e84491c0eda050ba924d7429,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,NaN,10000.000,HBT,0x0ace32f6e87ac1457a5385f8eb0208f37263b415,erc20,9300.164784,5.370000e-08,412836.0,283366,NaN,NaN,NaN,0,NaN,V3/0xhabitat001.csv
1,0x2b6a267fbd6616cba3ef055ec53bcf658c36d44be690...,NaN,2021-05-17 11:36:11,0xc97f82c80df57c34e84491c0eda050ba924d7429,0xf0a3da0e045fdac0ffec2285d360e706586809e6,NaN,4000.000,DAI,0x6b175474e89094c44da98b954eedeac495271d0f,erc20,4013.530044,5.500000e-08,172764.0,115759,NaN,NaN,NaN,0,NaN,V3/0xhabitat001.csv
2,0x2b6a267fbd6616cba3ef055ec53bcf658c36d44be690...,NaN,2021-05-17 11:36:11,0xc97f82c80df57c34e84491c0eda050ba924d7429,0x3336318c7cd22d0a32477235637f779e7071459b,NaN,2500.000,DAI,0x6b175474e89094c44da98b954eedeac495271d0f,erc20,2508.456277,5.500000e-08,172764.0,115759,NaN,NaN,NaN,0,NaN,V3/0xhabitat001.csv
3,0x2b6a267fbd6616cba3ef055ec53bcf658c36d44be690...,NaN,2021-05-17 11:36:11,0xc97f82c80df57c34e84491c0eda050ba924d7429,0x0b5a2a20ed3773aa33274f6684e97810338c09cb,NaN,2500.000,DAI,0x6b175474e89094c44da98b954eedeac495271d0f,erc20,2508.456277,5.500000e-08,172764.0,115759,NaN,NaN,NaN,0,NaN,V3/0xhabitat001.csv
4,0x572ccb64ed02bf86d8826a6c47ecbe6fecbe754539c6...,NaN,2021-06-09 10:15:02,0xc97f82c80df57c34e84491c0eda050ba924d7429,0xdf708717070981a8097912318c722f19eefb3bef,NaN,41702.342,HBT,0x0ace32f6e87ac1457a5385f8eb0208f37263b415,erc20,79306.986250,1.400000e-08,103918.0,68492,NaN,NaN,NaN,0,NaN,V3/0xhabitat001.csv


In [5]:
df = data.groupby(['file_name'])['label'].sum().reset_index(name = 'n_attacks')  
df.loc[df['n_attacks'] == 1, 'sngl_mlt'] = 'single'
df.loc[df['n_attacks'] > 1, 'sngl_mlt'] = 'multi'
df = df[['file_name' , 'sngl_mlt']]



In [6]:
#case_split = cases.loc[:, ["File Names", "train/test splits"]]
#case_split.columns = ["file_name", "experiment_part"]
#case_split["file_name"] += ".csv"


case_split = cases.loc[:, ["File Names", "train/test splits"]]#cases.loc[:, ["File Names", "train/test splits"]]
case_split.columns = ["file_name", "experiment_part"]
case_split["file_name"] += ".csv"
case_split["file_name"] = 'V3/' + case_split['file_name'].astype(str)#case_split["file_name"] = 'V3/' + case_split['file_name'].astype(str)

In [7]:
data.shape

(951863, 20)

In [8]:
data = data.merge(case_split)

In [9]:
data.shape

(951863, 21)

In [10]:
data = data.merge(df)

In [11]:
data.shape

(951863, 22)

In [12]:
data["experiment_part"].value_counts()

Train         697954
Validation    138322
Test          115587
Name: experiment_part, dtype: int64

In [13]:
data['sngl_mlt'].value_counts()

multi     715836
single    236027
Name: sngl_mlt, dtype: int64

## Configure featrure engineering

In [14]:
from cyvers_ai_ds.features import AssetCentricFEPipeline
from cyvers_ai_ds.features.transaction import DEFAULT_AGG_LIST

In [15]:
pipe = AssetCentricFEPipeline()

In [16]:
out_data = pipe.execute(data)

In [17]:
tx_df, asset_df = out_data

In [18]:
tx_df.shape

(811733, 114)

In [19]:
tx_df.columns

Index(['transaction_id', 'sender_id', 'block_time', 'gas_price', 'gas_limit',
       'gas_used', 'snd_rcv_tkn_type_cnt_tx_min',
       'snd_rcv_tkn_type_cnt_tx_max', 'snd_rcv_tkn_type_cnt_tx_median',
       'snd_rcv_tkn_type_cnt_tx_mean',
       ...
       'snd_rcv_mean_time_diff_sec_tx_std_log_to_median_ratio',
       'snd_rcv_mean_time_diff_sec_tx_sum_log_to_median_ratio',
       'amount_usd_tx_min_log_to_median_ratio',
       'amount_usd_tx_max_log_to_median_ratio',
       'amount_usd_tx_median_log_to_median_ratio',
       'amount_usd_tx_mean_log_to_median_ratio',
       'amount_usd_tx_std_log_to_median_ratio',
       'amount_usd_tx_sum_log_to_median_ratio',
       'receiver_type_tx_nunique_log_to_median_ratio',
       'internal_tx_cnt_log_to_median_ratio'],
      dtype='object', length=114)

In [20]:
tx_metadata = data.loc[
    :, ["transaction_id", "file_name", "experiment_part", "sngl_mlt"]
].drop_duplicates()

In [21]:
tx_metadata.shape

(813001, 4)

In [22]:
tx_df = tx_df.merge(tx_metadata)

In [23]:
tx_df.shape

(813001, 117)

In [24]:
tx_df["experiment_part"].value_counts()


Train         651040
Validation    112755
Test           49206
Name: experiment_part, dtype: int64

In [25]:
tx_df["sngl_mlt"].value_counts()


multi     614852
single    198149
Name: sngl_mlt, dtype: int64

In [26]:
tx_df.loc[tx_df["experiment_part"] == "Train", :].to_csv(
    "data/train_data.csv", index=False
)
tx_df.loc[tx_df["experiment_part"] == "Validation", :].to_csv(
    "data/validation_data.csv", index=False
)
tx_df.loc[tx_df["experiment_part"] == "Test", :].to_csv(
    "data/test_data.csv", index=False
)

Back up data to s3

In [ ]:
from utils import MODEL_NAME
from utils.s3 import MODELS_BUCKET
import boto3

In [ ]:
bucket = boto3.resource('s3').Bucket(MODELS_BUCKET)
bucket.upload_file('data/train_data.csv', MODEL_NAME + '/data/train_data.csv')
bucket.upload_file('data/validation_data.csv', MODEL_NAME + '/data/validation_data.csv')
bucket.upload_file('data/test_data.csv', MODEL_NAME + '/data/test_data.csv')